# Support Vector Machine Learning Model (94.5% variance - F1 Macro)

Import required packages

In [1]:
import numpy as np # for multi-dimensional array operations
import pandas as pd # for reading data from .csv files
from sklearn.svm import SVC # for support vector machine model
from sklearn.decomposition import PCA # for principle component analysis (dimensionality reduction)
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, StratifiedKFold # for getting the best hyper parameters

Assign the training set and testing set to variables for easy reference

In [2]:
train_set = pd.read_csv('../train_tfidf_features.csv') # import the training set
test_set = pd.read_csv('../test_tfidf_features.csv') # import the testing set

Principal Component Analysis for train_set (94.5% variance)

In [3]:
train_set_label = train_set.loc[:, ["label"]]
features_names = [str(i) for i in range(0, 5000)]
train_set_features = train_set.loc[:, features_names] # train_set_features will not contain the label and id columns

# perform PCA
pca = PCA(n_components = 0.945)
train_set_reduced = pca.fit_transform(train_set_features)
train_set_reduced = pd.DataFrame(data = train_set_reduced)
train_set_reduced

,0,1,2,3,4,5,6,7,8,9,...,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.019164,-0.008955,-0.001930,-0.010720,0.006710,0.008641,-0.022763,-0.004247,-0.012213,0.012635
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.003634,-0.001672,-0.000871,0.003293,-0.003623,-0.003929,-0.004200,0.004458,-0.000444,0.001474
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.014918,-0.004324,0.000843,0.008819,-0.007350,0.000881,0.006524,-0.000423,0.001398,-0.004802
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,0.003906,0.005378,0.010883,-0.006075,0.019991,-0.001389,-0.002098,-0.003110,-0.028727,0.002654
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.003396,0.003044,0.010839,0.002384,-0.001125,0.009578,0.006493,0.006420,0.004724,-0.006210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.003981,-0.004687,0.005902,0.014240,0.005527,0.001685,-0.003517,-0.009550,-0.002820,0.005572
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.005988,-0.000750,0.005305,-0.004071,0.003895,0.005624,0.004684,-0.004236,0.015578,0.000182
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.004333,0.001282,0.001955,-0.010081,0.014075,0.011552,0.003144,-0.000601,-0.004691,0.018184
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.000585,-0.007636,0.001234,0.005300,0.000494,0.001086,0.001967,0.001109,0.001859,-0.000266


In [4]:
X = train_set_reduced
y = train_set_label

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20) # Train a SVC model using different kernel
X_train = X
y_train = y

In [5]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350
0,-0.083198,-0.016048,-0.010595,-0.001965,-0.013779,-0.010989,-0.009680,-0.007520,-0.022370,-0.023418,...,-0.019164,-0.008955,-0.001930,-0.010720,0.006710,0.008641,-0.022763,-0.004247,-0.012213,0.012635
1,-0.068421,-0.043649,-0.018443,-0.008228,-0.000051,-0.043177,0.127322,0.010281,0.014349,-0.005608,...,0.003634,-0.001672,-0.000871,0.003293,-0.003623,-0.003929,-0.004200,0.004458,-0.000444,0.001474
2,-0.080171,-0.044642,-0.015342,-0.008697,-0.010481,-0.057562,0.075615,0.115411,0.111499,0.070551,...,-0.014918,-0.004324,0.000843,0.008819,-0.007350,0.000881,0.006524,-0.000423,0.001398,-0.004802
3,0.028600,-0.040406,0.002784,0.007087,0.000462,-0.031219,-0.137443,0.114581,-0.013427,0.077351,...,0.003906,0.005378,0.010883,-0.006075,0.019991,-0.001389,-0.002098,-0.003110,-0.028727,0.002654
4,0.255054,-0.113418,-0.019237,-0.021468,-0.040033,-0.007000,-0.035960,0.008656,-0.026048,-0.003353,...,0.003396,0.003044,0.010839,0.002384,-0.001125,0.009578,0.006493,0.006420,0.004724,-0.006210
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17179,0.075930,0.078879,0.167305,-0.099091,0.304845,0.037762,-0.022524,0.018488,-0.015024,-0.006992,...,-0.003981,-0.004687,0.005902,0.014240,0.005527,0.001685,-0.003517,-0.009550,-0.002820,0.005572
17180,-0.072104,-0.019731,-0.014655,-0.005451,-0.005682,-0.012314,0.005986,-0.009017,-0.003207,-0.014674,...,-0.005988,-0.000750,0.005305,-0.004071,0.003895,0.005624,0.004684,-0.004236,0.015578,0.000182
17181,0.002079,-0.041786,-0.016886,-0.008919,-0.018063,-0.020858,0.006685,-0.006916,-0.017607,0.010785,...,0.004333,0.001282,0.001955,-0.010081,0.014075,0.011552,0.003144,-0.000601,-0.004691,0.018184
17182,0.091355,-0.055903,-0.008473,-0.017559,-0.002191,-0.013549,-0.012348,-0.003722,-0.017070,-0.010729,...,-0.000585,-0.007636,0.001234,0.005300,0.000494,0.001086,0.001967,0.001109,0.001859,-0.000266


In [6]:
y_train

,label
0,1
1,0
2,1
3,0
4,1
...,...
17179,0
17180,0
17181,1
17182,1


Principal Component Analysis for test_set (94.5% variance)

In [7]:
features_names = [str(i) for i in range(0, 5000)]
test_set_features = test_set.loc[:, features_names] # test_set_features will not contain the label and id columns

# perform PCA
test_set_reduced = pca.transform(test_set_features) # use the pca from the train_set?
test_set_features = pd.DataFrame(data = test_set_reduced)
test_set_features

,0,1,2,3,4,5,6,7,8,9,...,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350
0,-0.082801,-0.009343,-0.013543,-0.013317,-0.007984,-0.002459,-0.026278,0.004096,-0.014072,-0.020606,...,0.003811,-0.001635,-0.013836,-0.006009,-0.004903,0.003960,-0.006141,0.000173,0.004623,0.001753
1,-0.075595,0.005040,-0.017771,-0.014935,-0.000467,-0.001601,-0.055948,-0.017461,-0.031185,-0.017278,...,-0.010147,-0.015496,0.012552,-0.000346,-0.001328,-0.002670,-0.007322,-0.004605,0.009372,-0.000046
2,0.163876,0.059162,-0.057706,-0.040884,-0.063164,-0.003223,0.020473,-0.005967,-0.005539,-0.008680,...,-0.017877,0.003140,0.002554,-0.009860,-0.012160,0.005146,0.006717,0.003619,-0.002485,-0.012671
3,-0.006835,0.189357,-0.044389,-0.024452,-0.043835,-0.013077,0.001344,0.000674,-0.005021,-0.028871,...,0.001786,-0.001135,-0.003135,0.001039,0.004999,0.001868,-0.002420,-0.002208,-0.002674,0.005252
4,0.120704,-0.089485,0.015896,0.030057,-0.018015,-0.057111,-0.072387,0.053490,-0.153091,0.026020,...,0.003219,-0.000031,0.000466,0.002824,0.001698,-0.003681,-0.002522,0.005954,0.001539,0.000641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291,-0.076808,-0.024913,-0.014752,-0.008106,-0.009817,-0.013137,0.026288,-0.012828,-0.004845,-0.010727,...,0.001955,0.003516,-0.003010,-0.001716,-0.008306,0.003141,0.004038,0.006129,0.000582,-0.002163
4292,0.069331,-0.055078,-0.008249,-0.022183,-0.000882,0.024624,-0.085988,-0.152147,0.159634,0.018128,...,0.006021,-0.005932,-0.004768,0.005075,-0.011846,-0.007226,-0.007964,0.003533,-0.006295,0.005698
4293,0.042192,-0.056123,-0.009015,-0.022608,-0.000348,0.011589,-0.046442,-0.129357,0.126640,0.010080,...,-0.006763,0.010700,0.007042,-0.002628,0.015577,-0.001242,0.003638,-0.019262,-0.022405,0.019342
4294,0.220417,-0.101562,-0.013420,-0.014971,-0.033857,-0.007979,-0.018445,-0.004240,-0.019587,-0.000660,...,0.019311,0.009548,0.009971,0.005116,-0.000390,-0.004763,-0.001248,-0.001910,0.000324,-0.002370


Tuning the hyper-parameters and training the model based on the best hyper-parameters

In [8]:
# hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['linear', 'poly', 'sigmoid', 'rbf']} # initialise the hyper-parameters
hyper_parameters = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001], 'kernel': ['rbf']} # initialise the hyper-parameters
kfold = StratifiedKFold(n_splits = 3, shuffle = True, random_state = 0) # for 3-fold cross validation
# grid = GridSearchCV(SVC(), hyper_parameters, refit = True, verbose = 2) # cretae a GridSearchCV object to git to the taining data
grid = GridSearchCV(SVC(), param_grid = hyper_parameters, scoring = 'f1_macro', refit = 'f1_macro', n_jobs = 1 , cv = kfold, verbose = 2)
grid.fit(X_train, np.ravel(y_train)) # training the model using the best hyper-parameters
print(grid.best_params_) # gets the best hyper-parameters for SVM

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.9min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 7.4min
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time= 8.2min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 5.9min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 6.3min
[CV] END .......................C=0.1, gamma=0.1, kernel=rbf; total time= 6.1min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 6.0min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.9min
[CV] END ......................C=0.1, gamma=0.01, kernel=rbf; total time= 5.8min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.6min
[CV] END .....................C=0.1, gamma=0.001, kernel=rbf; total time= 5.5min
[CV] END .....................C=0.1, gamma=0.001

Predicting the labels for the test dataset based on the model with the best hyper-parameters

In [9]:
y_predicted = grid.predict(test_set_features)

In [10]:
# y_predicted = svc_model.predict(test_set_features)
y_predicted = pd.DataFrame(y_predicted, columns = ['label']) # convert y_predicted from nparray to pandas dataframe
y_predicted.insert(loc = 0, column = 'id', value = [i for i in range(17185, 17185 + 4296)]) # insert a column of the ids, starting from 17185
y_predicted.to_csv('skynet_submission_94_5_f1macro.csv', index = False) # output the predicted labels to ./skynet_submission_94_5_f1macro.csv